In [1]:
%pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install keras

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
import re  # For cleaning text

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ishik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ishik\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ishik\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\ishik\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\ishik\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\ishik\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\ishik\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\ishik\AppData\Roaming\nltk_data...
[

True

In [6]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']

# Load intents JSON with UTF-8 encoding
with open('intents.json', encoding='utf-8') as file:
    intents = json.load(file)

In [7]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Clean and tokenize each word in the pattern
        pattern = re.sub(r'[^\w\s]', '', pattern)  # Remove special characters
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [8]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

# Save words and classes as pickle files
with open('words.pkl', 'wb') as file:
    pickle.dump(words, file)
with open('classes.pkl', 'wb') as file:
    pickle.dump(classes, file)


127 documents
23 classes ['anger', 'anxiety', 'boost_self_esteem', 'cope_with_emotions', 'emotions_negative', 'emotions_positive', 'exams', 'explore_anger', 'explore_anxiety', 'explore_causes', 'explore_emotions', 'family_issues', 'goodbye', 'greeting', 'how_you_feel', 'motivation', 'normal_issues', 'relationships', 'self_doubt', 'study_tips', 'thanks', 'why_you_feel', 'work_stress']
153 unique lemmatized words ['a', 'about', 'amazing', 'anger', 'angry', 'annoyed', 'anxiety', 'anxious', 'anything', 'appreciate', 'are', 'argument', 'at', 'bad', 'better', 'bored', 'bye', 'calm', 'can', 'cant', 'care', 'catch', 'cheerful', 'come', 'conflict', 'control', 'cope', 'day', 'deadline', 'deal', 'depressed', 'didnt', 'distant', 'do', 'dont', 'doubt', 'down', 'energy', 'enough', 'evening', 'exam', 'excited', 'failed', 'failure', 'family', 'fear', 'feel', 'feeling', 'fight', 'focus', 'friend', 'from', 'frustrated', 'give', 'go', 'good', 'goodbye', 'grade', 'great', 'greeting', 'had', 'handle', 'hap

In [9]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in doc[0]]
    bag = [1 if w in pattern_words else 0 for w in words]
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([np.array(bag), np.array(output_row)])

# Shuffle and convert to numpy arrays
random.shuffle(training)
training = np.array(training, dtype=object)

# Split features and labels
train_x = list(training[:, 0])
train_y = list(training[:, 1])

print("Training data created")

Training data created


In [10]:
from keras.layers import Input  # Add this import

model = Sequential()
model.add(Input(shape=(len(train_x[0]),)))  # Ensure Input is properly used
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile the model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Train the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5')

print("Model created")


Epoch 1/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0310 - loss: 3.1343    
Epoch 2/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1379 - loss: 3.0527     
Epoch 3/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1372 - loss: 3.0166 
Epoch 4/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1102 - loss: 2.9362     
Epoch 5/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1383 - loss: 2.9162 
Epoch 6/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1193 - loss: 2.9218     
Epoch 7/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1854 - loss: 2.7252 
Epoch 8/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2315 - loss: 2.8003 
Epoch 9/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2444 - loss: 2.7118 
Epoch 10/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2777 - loss: 2.6486 
Epoch 11/200
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2902 - loss: 2.6428 
Epoch 12/200
26/26 ━━━━━━━━━━━━━━━━━━

Model created


In [11]:
from keras.models import load_model
model = load_model('chatbot_model.h5')

with open('intents.json', encoding='utf-8') as file:
    intents = json.load(file)
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))


In [12]:
def clean_up_sentence(sentence):
    # Clean and tokenize input sentence
    sentence = re.sub(r'[^\w\s]', '', sentence)  # Remove special characters
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"found in bag: {w}")
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    for i in intents_json['intents']:
        if i['tag'] == tag:
            return random.choice(i['responses'])

def chatbot_response(msg):
    try:
        ints = predict_class(msg, model)
        return getResponse(ints, intents)
    except Exception as e:
        print(f"Error: {e}")
        return "I'm sorry, I couldn't understand that."


In [ ]:
import tkinter
from tkinter import *
import pickle
from keras.models import load_model
import nltk
import json
import random

# Load the model and other required files
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json', encoding='utf-8').read())
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

# Function to clean up the sentence and create bag of words
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

# Function to handle message sending in the GUI
def send():
    msg = EntryBox.get("1.0", 'end-1c').strip()
    EntryBox.delete("0.0", END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12))

        # Get the bot response
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

# Create GUI window
base = Tk()
base.title("Akishi")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

# Create chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial")
ChatLog.config(state=DISABLED)

# Bind scrollbar to chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

# Create send button
SendButton = Button(base, font=("Verdana", 12, 'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b", fg='#ffffff',
                    command=send)

# Create message entry box
EntryBox = Text(base, bd=0, bg="white", width="29", height="5", font="Arial")

# Place components on the screen
scrollbar.place(x=376, y=6, height=386)
ChatLog.place(x=6, y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
